### Word2Vec
##### Feb 18th 2023
##### William Lu

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('punkt')  
import re
#from bs4 import BeautifulSoup
import contractions
import math

[nltk_data] Downloading package wordnet to
[nltk_data]     /home1/wlu98761/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home1/wlu98761/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path  = 'amazon_reviews_us_Beauty_v1_00.tsv'
df = pd.read_csv(path,on_bad_lines='skip',sep='\t')
df.head()

/tmp/SLURM_13907457/ipykernel_8491/3686755405.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path,on_bad_lines='skip',sep='\t')


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0.0,0.0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0.0,0.0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5,0.0,0.0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372,R3KSZHPAEVPEAL,B002HWS7RM,255651889,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5,0.0,0.0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407,RAI2OIG50KZ43,B00SM99KWU,116158747,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31


In [3]:
df2 = df.copy()
dff = df2[['review_body','star_rating']]
#dff['star_rating'] = dff['star_rating'].astype(int)
class_one_score1 = dff.loc[dff['star_rating']=='1']
class_one_score2 = dff.loc[dff['star_rating']=='2']
class_one_label = [class_one_score1,class_one_score2]
class_one =pd.concat(class_one_label,ignore_index=True)
class_one['class_label'] = '1'
class_two_label = dff.loc[dff['star_rating']=='3']
class_two = class_two_label.copy()
class_two['class_label'] = '2'
class_three_score1 = dff.loc[dff['star_rating']=='4']
class_three_score2 = dff.loc[dff['star_rating']=='5']
class_three_label = [class_three_score1,class_three_score2]
class_three =pd.concat(class_three_label,ignore_index=True)
class_three['class_label'] = '3'

In [4]:
# randonly select 20000 rows for each class_label
one = class_one.sample(n = 20000)
two = class_two.sample(n = 20000)
three = class_three.sample(n = 20000)

In [5]:
# combine the three dataframes together and form a balanced dataset
df_bal_class = [one,two,three]
df_bal = pd.concat(df_bal_class,ignore_index = True)
df_bal

,review_body,star_rating,class_label
0,The smell is terrible! I'm not sure if I can k...,1,1
1,Color doesn't stay on well & looks cheap,2,1
2,"Was small, smelled weird, and also was broken ...",1,1
3,I don't want this item to have a star it was h...,1,1
4,I really love this cream/serum! Right when you...,1,1
...,...,...,...
59995,The smell of this shampoo is amazing. I've tri...,4,3
59996,After a week I noticed a reduction in hair los...,5,3
59997,"GOOD VALUE, GOOD PRODUCT",5,3
59998,Works really well. I use it in the shower and ...,5,3


In [6]:
df_bal2 = df_bal.copy()

In [7]:
# basic data cleaning
df_bal['review_body'] = df_bal['review_body'].astype(str)
# convert all reviews to lowercase
df_bal['review_body']= df_bal['review_body'].str.lower()
# remove urls from the reviews
df_bal["review_body"] = df_bal["review_body"].str.replace(r'\s*https?://\S+(\s+|$)', '').str.strip()
# remove htmls tag from the reviews
df_bal['review_body'] = df_bal['review_body'].str.replace(r'<[^<>]*>', '', regex=True)
# remove numerics
df_bal['review_body'] = df_bal['review_body'].str.replace('\d+', '',regex=True)
# remove non-alphabetical characters
df_bal['review_body'] = df_bal['review_body'].str.replace('[^a-zA-Z0-9]', ' ', regex=True)
# perform contractions
df_bal['review_body'] = df_bal['review_body'].apply(lambda x: contractions.fix(x))
# remove extra space
df_bal['review_body'] = df_bal['review_body'].str.strip()

/tmp/SLURM_13907457/ipykernel_8491/3100350469.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_bal["review_body"] = df_bal["review_body"].str.replace(r'\s*https?://\S+(\s+|$)', '').str.strip()


In [8]:
df_bal

,review_body,star_rating,class_label
0,the smell is terrible i m not sure if i can k...,1,1
1,color doesn t stay on well looks cheap,2,1
2,was small smelled weird and also was broken ...,1,1
3,i don t want this item to have a star it was h...,1,1
4,i really love this cream serum right when you...,1,1
...,...,...,...
59995,the smell of this shampoo is amazing i ve tri...,4,3
59996,after a week i noticed a reduction in hair los...,5,3
59997,good value good product,5,3
59998,works really well i use it in the shower and ...,5,3


### 2. Word Embeddings using Gensim Libarary

Word2Vec is a more recent model that embeds words in a lower-dimensional vector space using a shallow neural network. The result is a set of word-vectors where vectors close together in vector space have similar meanings based on context, and word-vectors distant to each other have differing meanings. 

https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html

(a) Figuring out how to extract word embedding. Check semantic similarities of generated vectors.

In [9]:
import gensim.downloader as api
import gensim
from gensim.models import Word2Vec
from nltk import word_tokenize
wv = api.load('word2vec-google-news-300')

In [10]:
# more cleaning here to make sure no space between each word
df_text = df_bal["review_body"]
df_text = list(df_text)
df_text = [' '.join(text.split()) for text in df_text]

In [11]:
# tokenize the text to words so that we can do word2vec
df_text_tokenized = []
for text in df_text:
    temp = word_tokenize(text)
    df_text_tokenized.append(temp)

In [12]:
df_text_tokenized[1]

['color', 'doesn', 't', 'stay', 'on', 'well', 'looks', 'cheap']

In [13]:
w1 = "outstanding"
w2 = "excellent"
w3 = "car"
w4 = "train"
w5 = "university"
w6 = "college"
example1 = wv.similarity(w1, w2)
example2 = wv.similarity(w3, w4)
example3 = wv.similarity(w5, w6)
# example1_similar_words = wv.most_similar(positive=[w1,w2],topn=1)
# I could not use this most_similar since I'm using colab. It crashes and one post @528 says we can just use
# the wv.similarity
print(f"The semantic similarity between {w1} and {w2} is: ", example1)
print(f"The semantic similarity between {w3} and {w4} is: ", example2)
print(f"The semantic similarity between {w5} and {w6} is: ", example3)
# print(f"The most similar word compared to {w1} and {w2} is: ", example1_similar_words)

The semantic similarity between outstanding and excellent is:  0.55674857
The semantic similarity between car and train is:  0.3402561
The semantic similarity between university and college is:  0.6385269


reference: https://radimrehurek.com/gensim/models/word2vec.html


(b) Train a word2vec model

The text below is just for studying purposes.

"size: The number of dimensions of the embeddings and the default is 100.

window: The maximum distance between a target word and words around the target word. The default window is 5.

min_count: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 5.

workers: The number of partitions during training and the default workers is 3.

sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW."

https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

In [14]:
my_w2v = Word2Vec(df_text_tokenized,min_count=9,vector_size=300,window=13,workers=3,sg=1)
my_w2v.save("word2vec_amazon.model")

Check the semantic similarities for the 2 examples

In [15]:
model1 = gensim.models.Word2Vec.load("word2vec_amazon.model")

In [16]:
my_model_example1_similarity = model1.wv.similarity(w1,w2)
my_model_example2_similarity = model1.wv.similarity(w3,w4)
#my_model_example3_similarity = my_w2v.similarity(w5,w6)
print(f"The semantic similarity between {w1} and {w2} from my w2v model is: ", my_model_example1_similarity)
print(f"The semantic similarity between {w3} and {w4} from my w2v model is: ", my_model_example2_similarity)
#print(f"The semantic similarity between {w5} and {w6} from my w2v model is: ", my_model_example3_similarity)

The semantic similarity between outstanding and excellent from my w2v model is:  0.48570466
The semantic similarity between car and train from my w2v model is:  0.48843965


Now we use CBOW as the training algorithm.

In [17]:
my_w2v_2 = Word2Vec(df_text_tokenized,min_count=9,vector_size=300,window=13,workers=3,sg=0)
my_w2v_2.save("word2vec_amazon2.model")

In [18]:
model2 = gensim.models.Word2Vec.load("word2vec_amazon2.model")

In [19]:
my_model_example1_similarity2 = model2.wv.similarity(w1,w2)
my_model_example2_similarity2 = model2.wv.similarity(w3,w4)

In [20]:
print(f"The semantic similarity between {w1} and {w2} from my second w2v model is: ", my_model_example1_similarity2)
print(f"The semantic similarity between {w3} and {w4} from my second w2v model is: ", my_model_example2_similarity2)

The semantic similarity between outstanding and excellent from my second w2v model is:  0.67975754
The semantic similarity between car and train from my second w2v model is:  0.15224382


In [21]:
print(wv.most_similar('outstanding'))

[('oustanding', 0.8012188673019409), ('Outstanding', 0.6041857600212097), ('exceptional', 0.6031844615936279), ('anchorman_Jason_Lezak', 0.5947381258010864), ('outsanding', 0.566262423992157), ('Stock_HEI', 0.5573362708091736), ('excellent', 0.556748628616333), ('Synplicity_FPGA_implementation', 0.5520347356796265), ('exemplary', 0.5467386245727539), ('W3_Awards_honors', 0.5172522068023682)]


In [22]:
print(wv.most_similar_cosmul(positive= ['king','woman'],negative=['man'], topn= 1))

[('queen', 0.9314123392105103)]


Comparing the model generated by myself and the pretained model, I think my the pretrained model describes the similarity between two words better. I also conclude that not all the words from the pretained model are in the model I train. In other words, my trained features mainly are about reviews. Some words do not often appear in reveiws.

### 3. Use Google pre-trained W2V features to train a single perceptron model and an SVM model

Split the data into training and testing.

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
def vector_average(x,w):
    temp = np.zeros(300)
    n=0
    for word in x:
        if word in w:
            n+=1
            temp+=w[word]
    if n>0:
        return temp/n
    else:
        return temp

In [25]:
#df_bal["tokenized"]= df_bal.apply(lambda x: word_tokenize(x['review_body']), axis=1)
df_bal["cleaned_tokenized"] = df_text_tokenized

In [26]:
df_bal["vector"]=df_bal["cleaned_tokenized"].apply(lambda x:vector_average(x,wv))

In [27]:
df_bal["vector"]

0        [0.017755126953125, 0.019000244140625, 0.04739...
1        [-0.016933441162109375, 0.07696533203125, -0.1...
2        [0.06226399739583333, 0.025470542907714843, 0....
3        [0.01211270419034091, 0.07297784631902521, 0.0...
4        [0.01719908783401268, 0.026452299477397533, 0....
                               ...                        
59995    [0.03624439239501953, 0.034887611865997314, 0....
59996    [-0.029820033482142856, 0.07513776506696429, 0...
59997    [0.0247802734375, 0.05975341796875, 0.05206298...
59998    [-0.02181827320772059, 0.10931834052590762, 0....
59999    [0.02679443359375, 0.017664591471354168, 0.006...
Name: vector, Length: 60000, dtype: object

In [28]:
temp = 300
for i in df_bal["vector"]:
    if len(i)<temp:
        temp = len(i)
print(temp)

300


In [29]:
df_bal["class_label"] = pd.to_numeric(df_bal["class_label"])

Perceptron (pass in the features into the model)

In [30]:
X_train,X_test,y_train,y_test = train_test_split(list(df_bal["vector"]),df_bal["class_label"],test_size = 0.2,random_state=42)
y_train = y_train.values
y_test = y_test.values

In [31]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score
perceptron = Perceptron()
perceptron.fit(list(X_train),y_train)
y_pred_perceptron = perceptron.predict(X_test)

In [32]:
acc_score_w2v_perceptron = accuracy_score(y_pred_perceptron,y_test)
print("Accuracy score for the perceptron model using word2vec-google-news-300 features is: ",acc_score_w2v_perceptron)

Accuracy score for the perceptron model using word2vec-google-news-300 features is:  0.57025


In [33]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(X_train,y_train)
y_pred_svm = svm.predict(X_test)

In [34]:
acc_score_w2v_svm = accuracy_score(y_pred_svm,y_test)

In [35]:
print("Accuracy score for the SVM model using word2vec-google-news-300 features is: ",acc_score_w2v_svm)

Accuracy score for the SVM model using word2vec-google-news-300 features is:  0.6745


#### For using TF-IDF features, we can just report the accuracy from HW1 according to post @569 on Piazza.

In [36]:
print("Accuracy score for the perceptron model using TF-IDF features is: ",0.61)
print("Accuracy score for the SVM model using TF-IDF features is: ",0.67)

Accuracy score for the perceptron model using TF-IDF features is:  0.61
Accuracy score for the SVM model using TF-IDF features is:  0.67


So we can conclude that using word2vec features as input can help us achieve a higher accuracy score than using TF-IDF. Especially for the perceptron model.

### 4. Feedforward Neural Networks

Using the Word2Vec features, train a feedforward multilayer perceptron network
for classification. Consider a network with two hidden layers, each with
100 and 10 nodes, respectively. You can use cross entropy loss and your
own choice for other hyperparamters.

You can use cross entropy loss and your
own choice for other hyperparamters, e.g., nonlinearity, number of epochs

reference: https://www.kaggle.com/code/mishra1993/pytorch-multi-layer-perceptron-mnist/notebook

(a) Use the average Word2Vec vectors to generate the input features

In [37]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

/home1/wlu98761/.conda/envs/tweet_capture_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
import torch.nn as nn
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch import optim

In [39]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1,self).__init__()
        hidden_layer1 = 100
        hidden_layer2 = 10
        self.fc1 = nn.Linear(300,hidden_layer1)
        self.fc2 = nn.Linear(hidden_layer1,hidden_layer2)
        self.fc3 = nn.Linear(hidden_layer2,3)
        self.dropout = nn.Dropout(0.2)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [40]:
FNN = Net1()
print(FNN)

Net1(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [41]:
# Specify Loss Function and Optimizer
# Categorical cross-entropy
criterion = nn.CrossEntropyLoss()
# Optimizer 
optimizer = optim.Adam(FNN.parameters(),lr=0.001)

In [42]:
#X_train_FNN = torch.Tensor(X_train)
#X_test_FNN = torch.Tensor(X_test)
y_train_tensor = torch.LongTensor(y_train-1)
y_test_tensor = torch.LongTensor(y_test-1)

In [43]:
num_epoch = 50
for epoch in range(num_epoch):
    y_pred_FNN = FNN(torch.from_numpy(np.asarray(X_train)).float())
    loss_FNN = criterion(y_pred_FNN,y_train_tensor)
    print("Epoch {} \ttraining loss: {:.6f}".format(epoch+1,loss_FNN))
    optimizer.zero_grad()
    # backward pass
    loss_FNN.backward()
    optimizer.step()

Epoch 1 	training loss: 1.128162
Epoch 2 	training loss: 1.126405
Epoch 3 	training loss: 1.124433
Epoch 4 	training loss: 1.122028
Epoch 5 	training loss: 1.119618
Epoch 6 	training loss: 1.117301
Epoch 7 	training loss: 1.115233
Epoch 8 	training loss: 1.112937
Epoch 9 	training loss: 1.110993
Epoch 10 	training loss: 1.108657
Epoch 11 	training loss: 1.106557
Epoch 12 	training loss: 1.104561
Epoch 13 	training loss: 1.102927
Epoch 14 	training loss: 1.100927
Epoch 15 	training loss: 1.098952
Epoch 16 	training loss: 1.097641
Epoch 17 	training loss: 1.096733
Epoch 18 	training loss: 1.094750
Epoch 19 	training loss: 1.093542
Epoch 20 	training loss: 1.092620
Epoch 21 	training loss: 1.090882
Epoch 22 	training loss: 1.089286
Epoch 23 	training loss: 1.086766
Epoch 24 	training loss: 1.085494
Epoch 25 	training loss: 1.083333
Epoch 26 	training loss: 1.081580
Epoch 27 	training loss: 1.078220
Epoch 28 	training loss: 1.077083
Epoch 29 	training loss: 1.075523
Epoch 30 	training loss

In [44]:
y_pred_FNN = np.array(FNN(torch.from_numpy(np.asarray(X_test)).float()).argmax(axis=1))
counter = 0
for i in range(0,len(y_pred_FNN)):
    if y_pred_FNN[i]+1 ==y_test[i]:
        counter += 1 # find the number of correct predictions
print("The accuracy for Feedforward Neural Networks by using word2vec-google-news-300 is:", counter/len(y_pred_FNN))

The accuracy for Feedforward Neural Networks by using word2vec-google-news-300 is: 0.525


In [45]:
y_pred_FNN

array([1, 0, 0, ..., 2, 2, 1])

In [46]:
y_test

array([1, 2, 2, ..., 3, 2, 2])

(b) Concatethe first 10 Word2Vec vectors for each review to generate input features

In [56]:
df_text2 = df_bal["review_body"]
df_text2 = list(df_text2)
df_text2 = [' '.join(text.split()) for text in df_text]
df_bal["non space text"] = df_text2

In [57]:
df_bal["non space text"]

0        the smell is terrible i m not sure if i can ke...
1                   color doesn t stay on well looks cheap
2        was small smelled weird and also was broken ei...
3        i don t want this item to have a star it was h...
4        i really love this cream serum right when you ...
                               ...                        
59995    the smell of this shampoo is amazing i ve trie...
59996    after a week i noticed a reduction in hair los...
59997                              good value good product
59998    works really well i use it in the shower and a...
59999             amazing product personal dentist at home
Name: non space text, Length: 60000, dtype: object

In [58]:
X_train_10,X_test_10,Y_train_10,Y_test_10 = train_test_split(df_bal["non space text"],df_bal["class_label"].values, test_size = 0.2,random_state=42)

In [59]:
X_train_10

48572                                      amazing product
38696                                it s ok for the price
13611    came in early shiny blah blah it was broken an...
35213    i was looking for more like a nars orgasm knoc...
31766    this is soft and won t dry out your lips but i...
                               ...                        
54343                               what a fantastic scent
38158    eh nothing astoundingly delicious in this coll...
860                you must be kidding use for how you wen
15795    mah on my hunt for hair product that does not ...
56422    color turned out perfect and it did not ruin m...
Name: non space text, Length: 48000, dtype: object

In [126]:
def tokenize(temp):
# For each sentence, word tokenization is performed using NLTK
    result = [word_tokenize(sentence) for sentence in temp]
    return result

In [52]:
def word_embed_10(x, m):
    X=[]
    vecs =[]
    for w in x:
        if w in m:
            vecs.append(m[w])
    while len(vecs) < 10:
        vecs.append(np.zeros(300))
    X.append(np.concatenate(vecs[:10]))
    return X

In [60]:
df_text10 = list(X_train_10)
df_text10 = [' '.join(text.split()) for text in df_text10]
df_text_tokenized10_train = []
for text in df_text10:
    temp = word_tokenize(text)
    df_text_tokenized10_train.append(temp)

In [68]:
df_text_tokenized10_train_em=[word_embed_10(x,wv) for x in df_text_tokenized10_train]

In [70]:
df_text10_test = list(X_test_10)
df_text10_test = [' '.join(text.split()) for text in df_text10_test]
df_text_tokenized10_test = []
for text in df_text10_test:
    temp = word_tokenize(text)
    df_text_tokenized10_test.append(temp)

In [71]:
df_text_tokenized10_test_em=[word_embed_10(x,wv) for x in df_text_tokenized10_test]

In [83]:
Y_train_10

array([3, 2, 1, ..., 1, 1, 3])

In [84]:
Y_train_10_tensor

tensor([2, 1, 0,  ..., 0, 0, 2])

In [101]:
df_text_tokenized10_train_em

[[array([ 0.07373047,  0.00405884, -0.13574219, ...,  0.        ,
          0.        ,  0.        ])],
 [array([ 0.08447266, -0.00035286,  0.05322266, ...,  0.        ,
          0.        ,  0.        ])],
 [array([ 0.12597656,  0.12792969,  0.19628906, ..., -0.11425781,
         -0.0378418 , -0.15136719], dtype=float32)],
 [array([-0.22558594, -0.01953125,  0.09082031, ...,  0.01708984,
          0.06079102, -0.10888672], dtype=float32)],
 [array([ 0.109375  ,  0.140625  , -0.03173828, ..., -0.05566406,
          0.10498047, -0.10839844], dtype=float32)],
 [array([ 0.07177734,  0.20800781, -0.02844238, ...,  0.        ,
          0.        ,  0.        ])],
 [array([0.01159668, 0.21679688, 0.14746094, ..., 0.        , 0.        ,
         0.        ])],
 [array([ 0.12988281,  0.13183594, -0.03295898, ..., -0.12109375,
         -0.10058594,  0.28125   ], dtype=float32)],
 [array([-0.01831055,  0.05566406, -0.01153564, ...,  0.        ,
          0.        ,  0.        ])],
 [array([ 

In [100]:
df_text_tokenized10_test_em

[[array([ 0.20019531,  0.15429688,  0.10302734, ..., -0.06347656,
          0.02111816, -0.08251953], dtype=float32)],
 [array([ 0.14453125,  0.04711914,  0.10058594, ..., -0.06884766,
          0.00778198,  0.09765625], dtype=float32)],
 [array([ 0.08447266, -0.00035286,  0.05322266, ...,  0.00366211,
          0.04760742, -0.06884766], dtype=float32)],
 [array([-0.17675781,  0.02941895, -0.13769531, ..., -0.10888672,
          0.04882812, -0.13085938], dtype=float32)],
 [array([ 0.01660156,  0.0456543 , -0.11914062, ...,  0.        ,
          0.        ,  0.        ])],
 [array([-0.22558594, -0.01953125,  0.09082031, ...,  0.07128906,
         -0.03491211,  0.02416992], dtype=float32)],
 [array([-0.08251953,  0.02246094, -0.14941406, ..., -0.19824219,
         -0.12304688,  0.06494141], dtype=float32)],
 [array([-0.22558594, -0.01953125,  0.09082031, ..., -0.19726562,
         -0.01031494, -0.0255127 ], dtype=float32)],
 [array([ 0.09570312,  0.19335938, -0.1484375 , ...,  0.       

In [85]:
Y_train_10_tensor= torch.LongTensor(Y_train_10-1)
Y_test_10_tensor= torch.LongTensor(Y_test_10-1)

In [104]:
class Net_10(nn.Module):
    def __init__(self):
        super(Net_10, self).__init__()
        hidden_1 = 100
        hidden_2 = 10
        self.fc1 = nn.Linear(300*10, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, 3)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.view(-1, 300*10)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [105]:
model_FNN_10 = Net_10()
print(model_ternary3)


TernaryNet_10(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [108]:
num_epoch = 100
for epoch in range(num_epoch):
    y_pred_FNN_10 = model_FNN_10(torch.from_numpy(np.asarray(df_text_tokenized10_train_em)).float())
    loss_FNN_10 = criterion(y_pred_FNN_10,Y_train_10_tensor)
    print("Epoch {} \ttraining loss: {:.6f}".format(epoch+1,loss_FNN_10))
    optimizer.zero_grad()
    # backward pass
    loss_FNN_10.backward()
    optimizer.step()

Epoch 1 	training loss: 1.124599
Epoch 2 	training loss: 1.124601
Epoch 3 	training loss: 1.124624
Epoch 4 	training loss: 1.124595
Epoch 5 	training loss: 1.124610
Epoch 6 	training loss: 1.124606
Epoch 7 	training loss: 1.124615
Epoch 8 	training loss: 1.124616
Epoch 9 	training loss: 1.124602
Epoch 10 	training loss: 1.124603
Epoch 11 	training loss: 1.124589
Epoch 12 	training loss: 1.124602
Epoch 13 	training loss: 1.124598
Epoch 14 	training loss: 1.124576
Epoch 15 	training loss: 1.124602
Epoch 16 	training loss: 1.124610
Epoch 17 	training loss: 1.124584
Epoch 18 	training loss: 1.124615
Epoch 19 	training loss: 1.124600
Epoch 20 	training loss: 1.124617
Epoch 21 	training loss: 1.124622
Epoch 22 	training loss: 1.124607
Epoch 23 	training loss: 1.124630
Epoch 24 	training loss: 1.124595
Epoch 25 	training loss: 1.124612
Epoch 26 	training loss: 1.124586
Epoch 27 	training loss: 1.124602
Epoch 28 	training loss: 1.124591
Epoch 29 	training loss: 1.124583
Epoch 30 	training loss

In [120]:
y_pred_FNN_10 = np.array(model_FNN_10(torch.from_numpy(np.asarray(df_text_tokenized10_test_em)).float()).argmax(axis=1))
counter_10 = 0
for i in range(0,len(y_pred_FNN_10)):
    if y_pred_FNN_10[i]+1==Y_test_10_tensor[i]:
        counter_10 += 1 # find the number of correct predictions
print("The accuracy for Feedforward Neural Networks by using word2vec-google-news-300 and concatenating first 10 words is:", counter_10/6000)

The accuracy for Feedforward Neural Networks by using word2vec-google-news-300 and concatenating first 10 words is: 0.669


I can conclude that using average word2vec FNN is better. It is similar to SVM and Perceptron accuracy. If concatenating the first 10 words, the accuracy is close to using the average word2vec features.

### 5. Recurrent Neural Networks

reference: https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

(a) Train RNN for sentiment analysis. Hidden state size = 20

In [121]:
from torch.nn.utils.rnn import pad_sequence

In [122]:
def word_embed_2(review, m):
    doc = []
    n = 0 
    for r in review:
        if r in m:
            n += 1
            doc.append(m[r])
            if n==20:
                break
    while n!=20:
        doc.append(np.zeros(300))
        n += 1
    return doc

In [123]:
def word_embedding(review):
    doc = []
    for r in review:
        doc.append([wv.key_to_index[word] if word in wv.key_to_index else 0 for word in r])
    return doc

In [124]:
x_train3,x_test3,y_train3,y_test3 = train_test_split(df_bal["review_body"],df_bal["class_label"].values, test_size = 0.2,random_state=42)

In [127]:
x_train3_tok = tokenize(x_train3)
x_test3_tok = tokenize(x_test3)

In [136]:
def padding(x,length):
    for i, j in enumerate(x):
        if len(j) >length:
            x[i] = j[:length]
        elif len(j)<length:
            x[i] = j[:len(j)]+[0]*(length-len(j))
    return x

In [137]:
new_x_train3_tok_em = word_embedding(x_train3_tok)
new_x_test3_tok_em = word_embedding(x_test3_tok)

In [139]:
new_x_train3_tok_em = np.array(padding(new_x_train3_tok_em,20))
new_x_test3_tok_em = np.array(padding(new_x_test3_tok_em,20))

In [141]:
xx_train3 = torch.LongTensor(new_x_train3_tok_em)
xx_test3 = torch.LongTensor(new_x_test3_tok_em)
yy_train3 = torch.LongTensor(y_train3-1)
yy_test3 = torch.LongTensor(y_test3-1)

In [142]:
Train3 = TensorDataset(xx_train3,yy_train3)
Test3 = TensorDataset(xx_test3,yy_test3)

In [143]:
load_Train3 = DataLoader(Train3, batch_size=32,shuffle=True)
load_Test3 = DataLoader(Test3, batch_size =32, shuffle =False)

In [161]:
yy_test3

tensor([0, 1, 1,  ..., 2, 1, 1])

In [144]:
input_size = len(wv)+1
hidden_size = 20
num_layers = 20
output_size = 1
from tqdm import tqdm
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, class_num):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size,num_layers)
        self.rnn = nn.RNN(num_layers, hidden_size, batch_first=True, nonlinearity="relu")
        self.fc = nn.Linear(hidden_size, class_num)

    def forward(self, x):
        # initize the hidden layer
        embedded_x = self.embedding(x)
        out, _ = self.rnn(embedded_x)
        output = self.fc(out)
        return output

In [145]:
model_RNN =  RNN(input_size,num_layers,hidden_size,output_size)

In [146]:
model_RNN

RNN(
  (embedding): Embedding(3000001, 20)
  (rnn): RNN(20, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

In [150]:
loss_RNN = nn.CrossEntropyLoss()
optimizer_RNN = optim.Adam(model_RNN.parameters(),lr=0.001)

In [165]:
# help from peers
def training(model,epoch,batch_size,loader,optimizer,loss):
    model.train()
    loss_epoch= 0
    for i, j in loader:
        optimizer.zero_grad()
        output = model(i)
        loss_per_epoch = loss(output,j.reshape(1,batch_size).t())
        loss_per_epoch.backward()
        optimizer.step()
        loss_epoch+=loss_per_epoch.item()
    print("Accuracy:{:.6f}".format(loss_per_epoch.item()))

In [176]:
# help from peers
def testing(model,test_loader):
    model.eval() 
    correct = 0
    with torch.no_grad():
        for i, t in test_loader:
            output = model(i) 
            _, predicted = torch.max(output.data, 1)
            correct += predicted.eq(t.data.view_as(predicted)).sum() 

    data_num = len(test_loader.dataset)
    print('\nAccuracy with test data is {}'.format(correct / data_num))

In [167]:
for epoch in range(5):
    training(model_RNN,epoch,32,load_Train3,optimizer_RNN,loss_RNN)
testing(model_RNN, load_Test3)

Accuracy:0.952885
Accuracy:0.817868
Accuracy:1.027247
Accuracy:0.665553
Accuracy:0.946841


NameError: name 'data' is not defined

Please ignore the pink area. I move the testing to below. I don't want to erase the result since it takes too long to run this cell.

In [177]:
testing(model_RNN, load_Test3)


Accuracy with test data is 0.5632500052452087


refence: https://pytorch.org/docs/stable/generated/torch.t.html#:~:text=Expects%20input%20to%20be%20%3C%3D%202,input%2C%200%2C%201)%20.
https://discuss.pytorch.org/t/valueerror-expected-input-batch-size-324-to-match-target-batch-size-4/24498/9



(b) GRU

In [179]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, class_num):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size,num_layers)
        self.rnn = nn.GRU(num_layers, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, class_num)

    def forward(self, x):
        # initize the hidden layer
        embedded_x = self.embedding(x)
        out, _ = self.gru(embedded_x)
        output = self.fc(out)
        return output

In [180]:
model_GRU =  RNN(input_size,num_layers,hidden_size,output_size)

In [ ]:
for epoch in range(5):
    training(model_GRU,epoch,32,load_Train3,optimizer_RNN,loss_RNN)
testing(model_GRU, load_Test3)

(c) LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, class_num):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size,num_layers)
        self.rnn = nn.LSTM(num_layers, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, class_num)

    def forward(self, x):
        # initize the hidden layer
        embedded_x = self.embedding(x)
        out, _ = self.lstm(embedded_x)
        output = self.fc(out)
        return output

In [ ]:
model_LSTM =  LSTM(input_size,num_layers,hidden_size,output_size)

In [ ]:
for epoch in range(5):
    training(model_LSTM,epoch,32,load_Train3,optimizer_RNN,loss_RNN)
testing(model_LSTM, load_Test3)

## Please give some partial credits for questions answered. since I worked really hard to find out the answer. Computing source is also another trouble of mine, so some output cannot be prinred out. Please be lenient. Thanks.